## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [6]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="ethereum"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('ethereum_train_8_2.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,0.23180,0.24981,0.23085,0.24797,11970598,USD
1,2020-06-02,0.24801,0.25321,0.22795,0.23794,12686011,USD
2,2020-06-03,0.23790,0.24476,0.23384,0.24452,8151002,USD
3,2020-06-04,0.24453,0.24641,0.23668,0.24337,8749785,USD
4,2020-06-05,0.24337,0.24750,0.23970,0.24023,7754761,USD
...,...,...,...,...,...,...,...
580,2022-01-02,3.76567,3.85116,3.71833,3.83067,242553,USD
581,2022-01-03,3.83070,3.84950,3.68248,3.76424,338507,USD
582,2022-01-04,3.76462,3.88907,3.71524,3.78952,415813,USD
583,2022-01-05,3.78447,3.84644,3.41846,3.53586,582713,USD


Chuẩn hóa dữ liệu

In [7]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Close'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Close'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Close'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Close'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Close'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}_8_2_standardized.csv")
print(df)

           Date     Open     High      Low    Close   Volume Currency  \
20   2020-06-21  0.22887  0.23118  0.22738  0.22813  4102533      USD   
21   2020-06-22  0.22811  0.24639  0.22769  0.24325  7929329      USD   
22   2020-06-23  0.24324  0.24477  0.24094  0.24320  5163349      USD   
23   2020-06-24  0.24320  0.24923  0.23156  0.23475  9127950      USD   
24   2020-06-25  0.23475  0.23529  0.22799  0.23256  6475813      USD   
..          ...      ...      ...      ...      ...      ...      ...   
580  2022-01-02  3.76567  3.85116  3.71833  3.83067   242553      USD   
581  2022-01-03  3.83070  3.84950  3.68248  3.76424   338507      USD   
582  2022-01-04  3.76462  3.88907  3.71524  3.78952   415813      USD   
583  2022-01-05  3.78447  3.84644  3.41846  3.53586   582713      USD   
584  2022-01-06  3.53615  3.55019  3.30614  3.40777   701888      USD   

         H-L      O-C     SMA_7    SMA_14    SMA_21      SD_7     SD_21  
20   0.00380  0.00074  0.231087  0.235277  0.2378

Scale Miền giá trị lại

In [8]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Close']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[1.54120489e-03],
       [4.83724702e-03],
       [4.82634741e-03],
       [2.98431328e-03],
       [2.50691035e-03],
       [1.87691288e-03],
       [0.00000000e+00],
       [8.78508585e-04],
       [1.48452691e-03],
       [9.96224375e-04],
       [2.15594290e-03],
       [1.16843822e-03],
       [8.85048351e-04],
       [1.76355694e-03],
       [1.50850606e-03],
       [4.46884019e-03],
       [3.96527820e-03],
       [5.65253787e-03],
       [4.56911661e-03],
       [4.40344253e-03],
       [3.95437859e-03],
       [4.72825092e-03],
       [4.06991446e-03],
       [4.23776846e-03],
       [3.82794311e-03],
       [2.74452186e-03],
       [2.54832887e-03],
       [3.20230549e-03],
       [3.91296007e-03],
       [3.27860276e-03],
       [5.33426925e-03],
       [9.20363088e-03],
       [1.18326169e-02],
       [1.27024058e-02],
       [1.83680232e-02],
       [1.96345579e-02],
       [2.19605347e-02],
       [2.09054524e-02],
       [2.10929257e-02],
       [2.48096928e-02],


In [9]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [10]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}82.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 56s 48ms/step - loss: 0.0334
Epoch 2/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0096
Epoch 3/60
40/40 [==============================] - 2s 54ms/step - loss: 0.0073
Epoch 4/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0071
Epoch 5/60
40/40 [==============================] - 2s 47ms/step - loss: 0.0066
Epoch 6/60
40/40 [==============================] - 2s 47ms/step - loss: 0.0066
Epoch 7/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0048
Epoch 8/60
40/40 [==============================] - 2s 47ms/step - loss: 0.0053
Epoch 9/60
40/40 [==============================] - 2s 46ms/step - loss: 0.0051
Epoch 10/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0045
Epoch 11/60
40/40 [==============================] - 2s 46ms/step - loss: 0.0043
Epoch 12/60
40/40 [==============================] - 2s 49ms/step - loss: 0.0057
Epoch 13/60
40/40 [=================